# Lab 8 - Latent Dirichlet Allocation (Part 2)

(30 points)

In this part of the lab, you will use the Latent Dirichlet Allocation approach to analyze the latent topics of two different datasets. 

Beside this iPython notebook, you are also given:

1. ingredients.xls: Each row represents one Medieval European recipe (in terms of ingredient list). The first cell of each row denotes the number of ingredients in this row, but there may be errors (you should write code to handle such cases). 

### What to submit
* Completed IPYNB with answers to questions at the end
* For instructions on submission, please go to webpage

### Online Latent Dirichlet Allocation Code

You don't need to implement anything in this part, but should read it to understand it.  You will use it when analysing the two datasets.

In [38]:
import sys, re, time, string
import numpy as n
from scipy.special import gammaln, psi

n.random.seed(100000001)
meanchangethresh = 0.001

In [39]:
def dirichlet_expectation(alpha):
    if (len(alpha.shape) == 1):
        return(psi(alpha) - psi(n.sum(alpha)))
    return(psi(alpha) - psi(n.sum(alpha, 1))[:, n.newaxis])

In [40]:
def parse_doc_list(docs, vocab):
    if (type(docs).__name__ == 'str'):
        temp = list()
        temp.append(docs)
        docs = temp

    D = len(docs)
    
    wordids = list()
    wordcts = list()
    for d in range(0, D):
        docs[d] = docs[d].lower()
        docs[d] = re.sub(r'-', ' ', docs[d])
        docs[d] = re.sub(r'[^a-z ]', '', docs[d])
        docs[d] = re.sub(r' +', ' ', docs[d])
        words = string.split(docs[d])
        ddict = dict()
        for word in words:
            if (word in vocab):
                wordtoken = vocab[word]
                if (not wordtoken in ddict):
                    ddict[wordtoken] = 0
                ddict[wordtoken] += 1
        wordids.append(ddict.keys())
        wordcts.append(ddict.values())

    return((wordids, wordcts))

In [41]:
class OnlineLDA:
    """
    Implements online Variational Bayes for LDA as described in (Hoffman et al. 2010).
    """

    def __init__(self, vocab, K, D, alpha, eta, tau0, kappa):
        """
        Arguments:
        K: Number of topics
        vocab: A set of words to recognize. When analyzing documents, any word
           not in this set will be ignored.
        D: Total number of documents in the population. For a fixed corpus,
           this is the size of the corpus. In the truly online setting, this
           can be an estimate of the maximum number of documents that
           could ever be seen.
        alpha: Hyperparameter for prior on weight vectors theta
        eta: Hyperparameter for prior on topics beta
        tau0: A (positive) learning parameter that downweights early iterations
        kappa: Learning rate: exponential decay rate---should be between
             (0.5, 1.0] to guarantee asymptotic convergence.

        Note that if you pass the same set of D documents in every time and
        set kappa=0 this class can also be used to do batch VB.
        """
        self._vocab = dict()
        for word in vocab:
            #word = word.lower()
            #word = re.sub(r'[^a-z]', '', word)
            self._vocab[word] = len(self._vocab)

        self._K = K
        self._W = len(self._vocab)
        self._D = D
        self._alpha = alpha
        self._eta = eta
        self._tau0 = tau0 + 1
        self._kappa = kappa
        self._updatect = 0

        # Initialize the variational distribution q(beta|lambda)
        self._lambda = 1*n.random.gamma(100., 1./100., (self._K, self._W))
        self._Elogbeta = dirichlet_expectation(self._lambda)
        self._expElogbeta = n.exp(self._Elogbeta)

    def do_e_step(self, docs):
        """
        Given a mini-batch of documents, estimates the parameters
        gamma controlling the variational distribution over the topic
        weights for each document in the mini-batch.

        Arguments:
        docs:  List of D documents. Each document must be represented
               as a string. (Word order is unimportant.) Any
               words not in the vocabulary will be ignored.

        Returns a tuple containing the estimated values of gamma,
        as well as sufficient statistics needed to update lambda.
        """
        # This is to handle the case where someone just hands us a single
        # document, not in a list.
        if (type(docs).__name__ == 'string'):
            temp = list()
            temp.append(docs)
            docs = temp

        (wordids, wordcts) = parse_doc_list(docs, self._vocab)
        batchD = len(docs)

        # Initialize the variational distribution q(theta|gamma) for
        # the mini-batch
        gamma = 1*n.random.gamma(100., 1./100., (batchD, self._K))
        Elogtheta = dirichlet_expectation(gamma)
        expElogtheta = n.exp(Elogtheta)

        sstats = n.zeros(self._lambda.shape)
        # Now, for each document d update that document's gamma and phi
        it = 0
        meanchange = 0
        for d in range(0, batchD):
            # These are mostly just shorthand (but might help cache locality)
            ids = wordids[d]
            cts = wordcts[d]
            gammad = gamma[d, :]
            Elogthetad = Elogtheta[d, :]
            expElogthetad = expElogtheta[d, :]
            expElogbetad = self._expElogbeta[:, ids]
            # The optimal phi_{dwk} is proportional to 
            # expElogthetad_k * expElogbetad_w. phinorm is the normalizer.
            phinorm = n.dot(expElogthetad, expElogbetad) + 1e-100
            # Iterate between gamma and phi until convergence
            for it in range(0, 100):
                lastgamma = gammad
                # We represent phi implicitly to save memory and time.
                # Substituting the value of the optimal phi back into
                # the update for gamma gives this update. Cf. Lee&Seung 2001.
                gammad = self._alpha + expElogthetad * \
                    n.dot(cts / phinorm, expElogbetad.T)
                Elogthetad = dirichlet_expectation(gammad)
                expElogthetad = n.exp(Elogthetad)
                phinorm = n.dot(expElogthetad, expElogbetad) + 1e-100
                # If gamma hasn't changed much, we're done.
                meanchange = n.mean(abs(gammad - lastgamma))
                if (meanchange < meanchangethresh):
                    break
            gamma[d, :] = gammad
            # Contribution of document d to the expected sufficient
            # statistics for the M step.
            sstats[:, ids] += n.outer(expElogthetad.T, cts/phinorm)

        # This step finishes computing the sufficient statistics for the
        # M step, so that
        # sstats[k, w] = \sum_d n_{dw} * phi_{dwk} 
        # = \sum_d n_{dw} * exp{Elogtheta_{dk} + Elogbeta_{kw}} / phinorm_{dw}.
        sstats = sstats * self._expElogbeta

        return((gamma, sstats))

    def update_lambda(self, docs):
        """
        First does an E step on the mini-batch given in wordids and
        wordcts, then uses the result of that E step to update the
        variational parameter matrix lambda.

        Arguments:
        docs:  List of D documents. Each document must be represented
               as a string. (Word order is unimportant.) Any
               words not in the vocabulary will be ignored.

        Returns gamma, the parameters to the variational distribution
        over the topic weights theta for the documents analyzed in this
        update.

        Also returns an estimate of the variational bound for the
        entire corpus for the OLD setting of lambda based on the
        documents passed in. This can be used as a (possibly very
        noisy) estimate of held-out likelihood.
        """

        # rhot will be between 0 and 1, and says how much to weight
        # the information we got from this mini-batch.
        rhot = pow(self._tau0 + self._updatect, -self._kappa)
        self._rhot = rhot
        # Do an E step to update gamma, phi | lambda for this
        # mini-batch. This also returns the information about phi that
        # we need to update lambda.
        (gamma, sstats) = self.do_e_step(docs)
        # Estimate held-out likelihood for current values of lambda.
        bound = self.approx_bound(docs, gamma)
        # Update lambda based on documents.
        self._lambda = self._lambda * (1-rhot) + \
            rhot * (self._eta + self._D * sstats / len(docs))
        self._Elogbeta = dirichlet_expectation(self._lambda)
        self._expElogbeta = n.exp(self._Elogbeta)
        self._updatect += 1

        return(gamma, bound)

    def approx_bound(self, docs, gamma):
        """
        Estimates the variational bound over *all documents* using only
        the documents passed in as "docs." gamma is the set of parameters
        to the variational distribution q(theta) corresponding to the
        set of documents passed in.

        The output of this function is going to be noisy, but can be
        useful for assessing convergence.
        """

        # This is to handle the case where someone just hands us a single
        # document, not in a list.
        if (type(docs).__name__ == 'string'):
            temp = list()
            temp.append(docs)
            docs = temp

        (wordids, wordcts) = parse_doc_list(docs, self._vocab)
        batchD = len(docs)

        score = 0
        Elogtheta = dirichlet_expectation(gamma)
        expElogtheta = n.exp(Elogtheta)

        # E[log p(docs | theta, beta)]
        for d in range(0, batchD):
            gammad = gamma[d, :]
            ids = wordids[d]
            cts = n.array(wordcts[d])
            phinorm = n.zeros(len(ids))
            for i in range(0, len(ids)):
                temp = Elogtheta[d, :] + self._Elogbeta[:, ids[i]]
                tmax = max(temp)
                phinorm[i] = n.log(sum(n.exp(temp - tmax))) + tmax
            score += n.sum(cts * phinorm)
#             oldphinorm = phinorm
#             phinorm = n.dot(expElogtheta[d, :], self._expElogbeta[:, ids])
#             print oldphinorm
#             print n.log(phinorm)
#             score += n.sum(cts * n.log(phinorm))

        # E[log p(theta | alpha) - log q(theta | gamma)]
        score += n.sum((self._alpha - gamma)*Elogtheta)
        score += n.sum(gammaln(gamma) - gammaln(self._alpha))
        score += sum(gammaln(self._alpha*self._K) - gammaln(n.sum(gamma, 1)))

        # Compensate for the subsampling of the population of documents
        score = score * self._D / len(docs)

        # E[log p(beta | eta) - log q (beta | lambda)]
        score = score + n.sum((self._eta-self._lambda)*self._Elogbeta)
        score = score + n.sum(gammaln(self._lambda) - gammaln(self._eta))
        score = score + n.sum(gammaln(self._eta*self._W) - 
                              gammaln(n.sum(self._lambda, 1)))

        return(score)

## Part 2a: The ingredient dataset

### Step 0: Load the packages

In [42]:
import cPickle, string, numpy, getopt, sys, random, time, re, pprint
from xlrd import open_workbook

### Step 1: Collect the vocabulary and the document lists 

    vocab: a list contains all the unique terms; (e.g. vocab[0] = "bean", vocab[1] = "broth")

    docset: a list contains all the receipts (e.g. docset[1] = "bean broth bacon")

In [43]:
wb = open_workbook('ingredients.xls')
ws = wb.sheet_by_name('ingredients')


In [44]:
vocab = []
docset = []
num_vocab = 0

#### 1. WRITE YOUR CODE HERE #### 
# (10 points)
# should be similar to the first code in Part 1 of this lab, 
# but NOT exactly the same. vocab is a list here, containing unique terms.
temp_list = []
for i in range(ws.nrows):
    num_words = ws.cell_value(i,0)
    for j in range(1,int(num_words)+1):
        if ws.cell_value(i,j) == '' or ws.cell_value(i,j) == None:
            continue
        if ws.cell_value(i,j) not in vocab:
            vocab.append(ws.cell_value(i,j))
            num_vocab += 1
        temp_list.append(ws.cell_value(i,j))
    result = ''
    for word in temp_list:
        result += word+' '
    docset.append(result)
    temp_list = []


In [45]:
print docset[0], len(vocab)

frumenty wheat oil broth milk almond saffron yolk egg  386


### Step 2: Decide the parameters needed
Decide the number of topics as you wish to recover from the document

In [46]:
K = 5

Decide the number of documents to analyze at each iteration

In [47]:
batchsize = 64

In [48]:
W = len(vocab)
D = len(docset)
documentstoanalyze = int(D/batchsize)

Initialize the algorithm with alpha=1/K, eta=1/K, tau_0=1024, kappa=0.7

In [49]:
# 2. Fill in function arguments for the Online LDA algorithm
# (5 points)
olda_ingredient = OnlineLDA(vocab, K, D, 1.0/K, 1.0/K, 1024, 0.7)

Decide the total number of data passes

In [50]:
n_pass = 1

### Step 3: Perform the online training

You are provided this code. You should run it and interpret the results later. 

The held-out perplexity is the most common way to evaluate a probabilistic model, by measuring the log-likelihood of a held-out test set.
 

In [51]:
print 'Iteration: Held-Out perplexity estimate'
for iteration in range(0, n_pass*documentstoanalyze):
    
    perm_indexes = numpy.random.permutation(range(D))[0:batchsize]
    #perm_indexes = range(iteration*batchsize, (iteration+1)*batchsize)
    sub_docset = []
    
    for idx in perm_indexes:
        sub_docset.append(docset[idx])
    
    # Give them to online LDA
    (gamma, bound) = olda_ingredient.update_lambda(sub_docset)
    
    # Compute an estimate of held-out perplexity
    (wordids, wordcts) = parse_doc_list(sub_docset, olda_ingredient._vocab)
    perwordbound = bound * len(sub_docset) / (D * sum(map(sum, wordcts)))
   
    print '%d:  %f' % \
        (iteration, numpy.exp(-perwordbound))

Iteration: Held-Out perplexity estimate
0:  1037.565911
1:  488.511976
2:  326.263653
3:  233.976321
4:  209.422146
5:  200.620250
6:  217.801180
7:  195.063699
8:  170.065494
9:  170.081268
10:  159.271088
11:  175.687720
12:  150.514830
13:  168.813606
14:  175.857747
15:  137.635929
16:  140.310316
17:  140.864815
18:  138.339653
19:  136.408191
20:  148.278282
21:  143.975898
22:  149.195784
23:  134.434910
24:  129.253549
25:  149.378892
26:  146.627100
27:  135.688642
28:  144.686519
29:  136.679787
30:  135.788388
31:  146.845143
32:  147.580788
33:  142.260414
34:  151.775968
35:  151.809844
36:  138.182309
37:  136.654602
38:  151.031837
39:  119.572652
40:  131.974619
41:  134.480431
42:  139.358076
43:  136.778725
44:  142.704234
45:  131.198295
46:  140.338717
47:  140.608894
48:  134.865832
49:  136.335010
50:  137.429220
51:  129.816195
52:  126.463670
53:  127.670785
54:  123.768791
55:  144.553437
56:  159.510529
57:  144.965538
58:  143.357466
59:  120.755181
60:  134.

### Step 4: Print the top K topics. 
Use the code below to print out top words in the K topics 

In [52]:
num_elements = 10

testlambda = olda_ingredient._lambda
for k in range(0, K):
    lambdak = list(testlambda[k, :])
    lambdak = lambdak / sum(lambdak)
    temp = zip(lambdak, range(0, len(lambdak)))
    temp = sorted(temp, key = lambda x: x[0], reverse=True)
    print 'topic %d:' % (k)
    for i in range(0, num_elements):
        print '%20s  \t---\t  %.4f' % (vocab[temp[i][1]], temp[i][0])

topic 0:
                salt  	---	  0.0509
              pepper  	---	  0.0466
                wine  	---	  0.0455
              ginger  	---	  0.0382
             vinegar  	---	  0.0376
               bread  	---	  0.0366
               broth  	---	  0.0341
             saffron  	---	  0.0297
               clove  	---	  0.0274
            cinnamon  	---	  0.0249
topic 1:
               crane  	---	  0.0303
               heron  	---	  0.0290
             peacock  	---	  0.0173
                plum  	---	  0.0145
                swan  	---	  0.0131
                bird  	---	  0.0114
            pheasant  	---	  0.0109
              cheese  	---	  0.0099
             bittern  	---	  0.0098
              garlic  	---	  0.0098
topic 2:
                bean  	---	  0.0326
                 oil  	---	  0.0194
              turnip  	---	  0.0188
                leek  	---	  0.0177
               onion  	---	  0.0170
                 pea  	---	  0.0166
              fennel  	---	  0.0163
 

## Questions: 
### (5+3+3+4 points)
   * Do you see any meaning in the top 5 topics you got? Can you identify the topics they are representing (for at least 2 of the 5)?
   * What do the numbers against every element signify?
   * What estimation method is used to determine the topics and the word distributions?
   * What are the alpha and eta parameters in the LDA algorithm?

## Your answers here
Each of the top five topics look like they represent a certain category within ingredients. For example, topic 1 looks to be of birds, and topic 0 looks like spices and condiments.

The numbers against every element looks like its the probability that the certain item appears within the certain topic.

The estimation method that is used to determine the topics and the word distributions is the Variational Bayes model.

Alpha and eta are parameters that affect the sparsity of theta and lambda distributions.